In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats

sample1 = pd.read_csv('sample_1.txt')
sample2 = pd.read_csv('sample_2.txt')
sample1 = sample1.values.reshape(-1)
sample2 = sample2.values.reshape(-1)

In [35]:
# H0: mean1 - mean2 == 0
# https://pl.wikipedia.org/wiki/Test_t_Welcha
mean_sample1 = np.mean(sample1)
mean_sample2 = np.mean(sample2)
var_sample1 = np.var(sample1)
var_sample2 = np.var(sample2)

t = (mean_sample1 - mean_sample2) / math.sqrt(var_sample1**2/len(sample1) + 
                                              var_sample2**2/len(sample2))
v1 = len(sample1) - 1 
v2 = len(sample2) - 1 
v_numerator = pow(var_sample1*var_sample1/len(sample1)+ var_sample2*var_sample2/len(sample2),2)
v_denominator = (pow(var_sample1, 4)/(pow(len(sample1),2)*v1) + pow(var_sample2, 4)/(pow(len(sample2),2)*v2))
v = v_numerator / v_denominator
print("t-statistics: "+str(t))
print("Degrees of Freedom: "+str(v))

t-statistics: -1.0640400422961207
Degrees of Freedom: 124.49277331619771


In [36]:
# https://pythonfordatascience.org/welch-t-test-python-pandas/
def welch_dof(x,y):
        dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
        print(f"Welch-Satterthwaite Degrees of Freedom = {dof:.4f}")
        
def welch_ttest(x, y): 
    ## Welch-Satterthwaite Degrees of Freedom ##
    dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
   
    t, p = stats.ttest_ind(x, y, equal_var = False)
    
#     print("\n",
#           f"Welch's t-test= {t:.4f}", "\n",
#           f"p-value = {p:.4f}", "\n",
#           f"Welch-Satterthwaite Degrees of Freedom= {dof:.4f}")
    return p

welch_ttest(sample1, sample2)

0.3155528729980055

In [30]:
# Test t-studenta używa się do oszacowania rozkładu prawdopodobieństwa wartości średniej w populacji na podstawie próby.
# Poziom istotności v równa się ilość elementów w próbie - 1. https://en.m.wikipedia.org/wiki/File:Student_t_pdf.svg
# Po co obliczyłem t?
# Na podstawie rozkładu studenta z wyliczonym parametrem v, muszę obliczyć jakie jest prawdopodobieństwo 
# że liczba wylosowana z tego rozkładu jest mniejsza od t lub większa od -t(two tailed test)
# https://www.khanacademy.org/math/ap-statistics/two-sample-inference/two-sample-t-test-means/v/two-sample-t-test-for-difference-of-means
# na podstawie wartości t i v wylicza się p-value, co dla 0.05 poziomu istotności pozwala nam nie odrzucić hipotezy zerowej


In [6]:
sample1_mean = 0.5
sample2_mean = 0.75
sample1_std = 1
sample2_std = 0.75

In [37]:
# sample1_distribution = np.random.normal(sample1_mean, sample1_std, 1000)
# sample2_distribution = np.random.normal(sample2_mean, sample2_std, 1000)
# plt.figure(figsize=(21,13))
# sample1_count, sample1_bins, sample1_ignored = plt.hist(sample1_distribution, 30, density=True)
# sample2_count, sample2_bins, sample2_ignored = plt.hist(sample2_distribution, 30, density=True)
# plt.clf()
# plt.plot(sample1_bins, 1/(sample1_std * np.sqrt(2 * np.pi)) *
#           np.exp( - (sample1_bins - sample1_mean)**2 / (2 * sample1_std**2) ),
#           linewidth=2, color='g')
# plt.plot(sample2_bins, 1/(sample2_std * np.sqrt(2 * np.pi)) *
#           np.exp( - (sample2_bins - sample2_mean)**2 / (2 * sample2_std**2) ),
#           linewidth=2, color='r')
# plt.show()

In [38]:
alpha = 0.05
under_alpha_count = 0
test_size = 1000
for i in range(test_size):
    sample1_distribution = np.random.normal(sample1_mean, sample1_std, len(sample1))
    sample2_distribution = np.random.normal(sample2_mean, sample2_std, len(sample2))
    p_value = welch_ttest(sample1_distribution, sample2_distribution)
#     if p_value > alpha:
#         print("WARNING")
#         print(p_value)
    if p_value < alpha:
        under_alpha_count += 1
test_power = (under_alpha_count / test_size)*100
print("For significance level of "+ str(alpha*100)+"%, the power of test is: "+str(test_power)+"%")

For significance level of 5.0%, the power of test is: 49.7


In [40]:
alpha = 0.01
test_size = 100
for sample_size in range(10, 1000):
    under_alpha_count = 0
    for i in range(test_size):
        sample1_distribution = np.random.normal(sample1_mean, sample1_std, sample_size)
        sample2_distribution = np.random.normal(sample2_mean, sample2_std, sample_size)
        p_value = welch_ttest(sample1_distribution, sample2_distribution)
        if p_value < alpha:
            under_alpha_count += 1
    test_power = (under_alpha_count / test_size)*100
    if test_power > 90:
        print("Minimum sample size: "+str(sample_size)+", test power: "+str(test_power)+", for significance level 1%")
        break


Minimum sample size: 366, test power: 90.3, for significance level 1%
